# Análise e classificação de produtos do marketplace.
### Classificação de produtos pela categoria e distinção de produtos únicos.

Neste notebook exploramos diferentes estratégias de ML/DL para classificar produtos em suas categorias, analisar o desempenho dos classificadores sobre base com milhares de róturos e dinstinguí-los entre suas correspondentes variantes.

In [ ]:
import os
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.spatial.distance import squareform
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from scipy.cluster.hierarchy import dendrogram, linkage, cut_tree, ward
#from sklearn.cluster import AgglomerativeClustering
from sklearn.base import clone
from sklearn.metrics import pairwise_distances

from gensim.models.fasttext import FastText

# Módulos locais

from utils.text_clean import clean_text
from utils.word2vec import Word2VecModel
from utils.doc2vec import Doc2VecModel

from utils.clfs import Clfs

In [ ]:
headers = ["p_id", "p_title", "vendor_id", "cluster_id", "cluster_title", "cat_id", "cat_title"]
df = pd.read_csv('datasets/pricerunner_aggregate.csv', header=0, names = headers)
#df = df[df.cat_id == 2612]
df.head()

# Construção de atributos (Features)

In [ ]:
df = df[ (df.p_title.notna()) & (df.p_title.notnull()) ]

In [ ]:
# Atribuindo a cada produto da base um inteiro com a quantidade de nomes
# diferentes estão associados ao seu ID (incluindo ele).
counts = df.cluster_id.value_counts()
freqs = [ counts[cid] for cid in df.cluster_id ]
df["freqs"] = freqs

In [ ]:
feat_tokens = clean_text(df.p_title.values)

In [ ]:
w2v_params = {
    "sentences": feat_tokens,
    "vector_size": 100,
    "sg": 0,
    "window": 6,
    "epochs": 10,
    "workers": 10
}

w2v = Word2VecModel(feat_tokens, params=w2v_params)

d2v_params = {
    "documents":  feat_tokens,
    "vector_size": 100,
    "dm": 0,
    "window": 6,
    "epochs": 10,
    "dbow_words": 1,
    "workers": 10
}

d2v = Doc2VecModel(feat_tokens, params=d2v_params)

In [ ]:
representacoes = {}
representacoes["word2vec"] = w2v.transform(feat_tokens)
representacoes["doc2vec"] = d2v.transform(len(feat_tokens))

# Classificação de produtos por categoria.

Verificando o desempenho dos classificadores por categoria.

In [ ]:
r = np.random
seed = r.randint(0, 2147483647 * 2)

classifiers = {
    "RandomForestClassifier": RandomForestClassifier(n_jobs=10, random_state=seed),
    "LogisticRegression": LogisticRegression(max_iter=400, multi_class='multinomial', n_jobs=10, random_state=seed),
    "GradientBoostingClassifier": GradientBoostingClassifier(random_state=seed),
    "KNeighborsClassifier": KNeighborsClassifier(n_jobs=10),
    "SVC": SVC(random_state=seed)
}

"""
classifiers = {
    "RandomForestClassifier": RandomForestClassifier(n_jobs=10, random_state=seed),
    "LogisticRegression": LogisticRegression(max_iter=2000, multi_class='multinomial', n_jobs=10, random_state=seed),
    "KNeighborsClassifier": KNeighborsClassifier(n_jobs=10)
}
"""

"""
classifiers = {
    "LogisticRegression": LogisticRegression(max_iter=2000, multi_class='multinomial', n_jobs=10, random_state=seed),
    "KNeighborsClassifier": KNeighborsClassifier(n_jobs=10)
}
"""



### Avaliando diferentes classificadores para classificação dos produtos em suas categorias.

In [ ]:
pipeline = Clfs()

results = []
scores_rep = {}
for rep in representacoes:
    scores_clfs = pipeline.fast_avaliation(classifiers, representacoes[rep], df.cat_id)
    for clf in scores_clfs:
        results.append([rep, clf, scores_clfs[clf]["mean_f1"], scores_clfs[clf]["std_f1"]])

cat_results = pd.DataFrame(results, columns=["Rep", "Clf", "MeanF1", "StdF1"])
timestamp = time.strftime("%Y-%m-%d - %H:%M:%S")
cat_results.to_csv(f'tables/f1_rep_{timestamp}.csv', index=False)
cat_results.head(10)

# Verificação dos classificadores para o problema PU.

### Análise do desempenho dos classificadores conforme mais produtos são adicionados a classificação.

In [ ]:

r = np.random
seed = r.randint(0, 2147483647 * 2)

# Verificando a taxa de acerto pela quantidade de classes.
#n_folds = 5
limit = 30
results = {}
for n_class in range(3, 11):
    ids = list(set(df[(df.freqs >= n_class) & (df.cat_id == 2622)].cluster_id))
    results[n_class] = {}
    for i in range(3, limit):
        results[n_class][i] = {}
        # Escolhendo um sample de produtos aleatório da base.
        clusters_ids = np.random.choice(ids, i)
        set_sample = df.cluster_id.isin(clusters_ids)
        # Para cada representação.
        for rep in representacoes:
            target = df[set_sample]["cluster_id"].values
            features = representacoes[rep][set_sample]
            results[n_class][i][rep] = pipeline.fast_avaliation(classifiers, features, target, n_folds=n_class)

from pprint import pprint

pprint(results)

In [ ]:

import json

with open('tables/f1_classes.json', 'w') as fd:
    json.dump(results, fd)

In [ ]:
def plot_graphs(results):

    total_class = len(results.keys())
    values_plot = {}

    # Para cada limite com classe.
    for n_class in results:
        values_plot[n_class] = {}
        # Para cada iteração.
        for it in results[n_class]:
            # Para cada representação.
            for rep in results[n_class][it]:
                if rep not in values_plot[n_class]:
                    values_plot[n_class][rep] = {}
                # Para cada classificador.
                for clf in results[n_class][it][rep]:
                    if clf not in values_plot[n_class][rep]:
                        values_plot[n_class][rep][clf] = []
                    mean_f1 = results[n_class][it][rep][clf]["mean_f1"]
                    values_plot[n_class][rep][clf].append(mean_f1)

    fig, axes = plt.subplots(4,2, figsize=(14,18))
    for n_class, index in zip(values_plot, range(total_class)):
        col = index % 2
        line = index // 2
        for rep in values_plot[n_class]:
            for clf in values_plot[n_class][rep]:
                y = values_plot[n_class][rep][clf]
                x = list(range(3, limit))
                label = rep+' '+clf
                axes[line, col].plot(x, y, label=label)
        axes[line, col].legend()
        axes[line, col].set_ylim((0,1))
        axes[line, col].set_xlabel("Number of Classes", fontsize=16)
        axes[line, col].set_ylabel("F1 (%)", fontsize=16)
        axes[line, col].set_title("Class with "+str(n_class), fontsize=18)
        axes[line, col].tick_params(axis='x', labelsize=16)
        axes[line, col].tick_params(axis='y', labelsize=16)
        axes[line, col].grid()
    fig.tight_layout()
    plt.savefig('graphs/f1_classes.pdf')


plot_graphs(results)


# Agrupamento de PUs por categoria

### Para cada categoria inicial da base de dados vamos agrupar os produtos únicos com a técnica de clustering aglomerativo. Utilizamos essa técnica isoladamente por categoria para que essa escale com maior facilidade conforme aumenta o volume dos dados.

In [ ]:
cat_ids = set(df.cat_id)

dists = {}
for cat_id in cat_ids:
    indexes = df.cat_id.isin([cat_id])
    feats = representacoes["word2vec"][indexes]
    dist = pairwise_distances(feats, metric="cosine", n_jobs=10)
    np.fill_diagonal(dist,0)
    dists[cat_id] = squareform(dist)

In [ ]:
def plot_dists(dists):

    cat_ids = list(dists.keys())
    total = len(cat_ids)
    fig, ax = plt.subplots(figsize=(14,10))
    for i in range(total):
        col = i % 2
        line = i // 2
        ax = plt.subplot(3, 4, i+1)
        x = list(range(dists[cat_ids[i]].shape[0]))
        y = np.copy(dists[cat_ids[i]])
        y.sort()
        label_cat = df[df.cat_id == cat_ids[i]].iloc[0]["cat_title"]
        plt.plot(x, y, label=label_cat)
        plt.grid()
        plt.xticks(fontsize=16)
        plt.yticks(fontsize=16)
        plt.xlabel("Comparasons", fontsize=16)
        plt.ylabel("Distances", fontsize=16)
        plt.tight_layout()
        plt.legend()

plot_dists(dists)
        
        

In [9]:
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment as linear_assignment

def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

def cluster_accuracy(labels, predicted_labels):
    cm = confusion_matrix(labels, predicted_labels)
    indexes = linear_assignment(_make_cost_m(cm))
    js = [e[1] for e in sorted(indexes, key=lambda x: x[0])]
    cm2 = cm[:, js]
    acc = np.trace(cm2) / np.sum(cm2)
    return acc


In [ ]:
for cat_id in dists:
    #clu = AgglomerativeClustering(affinity="precomputed", linkage="complete")
    #clu.fit(dists[cat_id])
    Z = ward(dists[cat_id])
    labels = cut_tree(Z, height=0.2)